[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/USeebi/jetsim-fork/blob/master/JETSIM_collect_images_colab.ipynb)

Jetsim : Collect Images in Colab
===



---

### INITIAL SETUP

Turn ON **Donkeycar Simulator** before running the code below

Setup **Cloudflare Tunnel** on the localhost running Donkeycar Simulator

Find the host computer name of the quick Tunnel created and substitute for SUBDOMAIN in the **cloudflared** access CLI below


### Step **0.0** from README.md
---


In [ ]:
# https://github.com/Triton-AI/jetsim

!pip install docopt num2words
!pip install git+https://github.com/tawnkramer/gym-donkeycar.git

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 6.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=cbfcb6662495d993735c9d3a69f61db58641962608b6b16ab8ad9c2e27406c3c
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
  Cloning https://github.com/tawnkramer/gym-donkeycar.git to /tmp/pip-req-build-lyoun5co
  Running command git clone --filter=blob:none --quiet https://github.com/tawnkramer/gym-donkeycar.git /tmp/pip-req-build-lyoun5co
  Resolved https://github.com/tawnkramer/gym-donkeycar.git to commit d6264fa46a93a50f6add282972efd8b70a18489e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Get

In [ ]:

!git clone https://github.com/USeebi/jetsim-fork.git


Cloning into 'jetsim'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 134 (delta 70), reused 89 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (134/134), 1.93 MiB | 14.75 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [ ]:
# Download and install cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb
!cloudflared -v

Selecting previously unselected package cloudflared.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.9.1) ...
Setting up cloudflared (2025.9.1) ...
Processing triggers for man-db (2.10.2-1) ...
cloudflared version 2025.9.1 (built 2025-09-22-13:28 UTC)


Change SUBDOMAIN to a host domain name before proceed.

In [ ]:
import subprocess

process = subprocess.Popen(
    [
        'bash', '-c',
        'cloudflared access tcp --hostname <SUBDOMAIN>.trycloudflare.com --url 127.0.0.1:9091 &'
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True  # Use text=True for string output
)

print(f"Process started with PID: {process.pid}")
#stdout_data, stderr_data = process.communicate()



Process started with PID: 12646


Create the racecar dictionary
===

---

customize your car settings.

Select track to train on ('warren' is defualt track)

---

In [ ]:
#initialize gym wrapper

GYM_DICT={
  'car':{
    #custom settings
    'racer_name': 'Jetsim_NAME',
    'bio' : 'Triton_AI_Jetsim = Jetracer',
    'country' : 'US',
    "guid": "GO_TRITON_AI",
    'body_style' : 'f1',         #car01 | f1 | donkey
    'body_rgb': (24, 43, 200),      #(R, G, B)
    'car_name' : 'Training_Jetsim', #displays above car
    'font_size' : 50,
    #camera settings
    "fov" : 80,
    "fish_eye_x" : 0.0,
    "fish_eye_y" : 0.0,
    "img_w" : 224,
    "img_h" : 224,
    "img_d" : 3,
    "img_enc" : 'JPG',
    "offset_x" : 0.0, #sides
    "offset_y" : 2.0, #height #Jetsim 2.0
    "offset_z" : 0.0, #forward
    "rot_x" : 20.0,   #tilt #Jetsim 0.0
    #"rot_y": 180,    #rotate
    },

    #connection type
    'default_connection': 'local',

    'local_connection':{
      'scene_name': 'warren', # warren | thunderhill | mini_monaco | roboracingleague_1 | generated_track | generated_road | warehouse | sparkfun_avc | waveshare
######YOUR HOST COMPUTER IP ADDRESS######
      'host': '127.0.0.1', #UPDATE WITH COMPUTER IP ADDRESS HOSTING SIM
######YOUR HOST COMPUTER IP ADDRESS######
      'port': 9091,
      'artificial_latency': 0},

    'remote_connection':{
        'scene_name': 'warren',
###### REMOTE IP ADDRESS ######
        'host': 'donkey-sim.roboticist.dev', # DIY Robocar virtual track server = 'donkey-sim.roboticist.dev'
###### REMOTE IP ADDRESS ######
        'port': 9091,
        'artificial_latency': 0},# Ping the remote simulator. millisecond here.

    'lidar':{
        'enabled': False,
        'deg_inc': 1, # Degree increment between each ray of the lidar
        'max_range': 30.0}, # Max range of the lidar laser
    }


Send dictionary to SdSandbox Gym interface
===

### Make sure to turn ON **Donkeycar Simulator** before running the code below

In [ ]:
import time
%cd jetsim-fork/jetsim_local/resources
from gyminterface import GymInterface
%cd ../../../

# Initialization of the GymInterface (inherited from SDClient) sets up a
# background thread to manage socket communication with the Donkeycar simulator.
gym = GymInterface(gym_config=GYM_DICT)
time.sleep(1)
imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
# https://blog.csdn.net/Karen_Yu_/article/details/115293733
#print(imageSeen.shape)
newArray = imageSeen[:]
print(newArray.shape)

# If you run this cell more than once it will generate more cars in the sim. Turn simulator application off and back on to delete all cars.


/content/jetsim/jetsim_local/resources
/content
Loading scene: warren
Sending configs...
Sending racer info
Car loaded.
Sending car config
Gym Interface: Camera resolution (224, 224).
(224, 224, 3)


---

This will initialize the **racecar in the simulator**. Check if the car **populated** in the sim.

When you call gym.step(...) the following happens internally:
- A control command is sent to the simulator.
- Your code waits for the next telemetry or relevant response, which is received and processed by on_msg_recv().
- on_msg_recv() updates the internal state, making the latest simulator data available for the next step or observation.

---

In [ ]:
# If the custom car setting did not load correctly, uncomment this to take car out of sim and then re-run the cell above.
#gym.onShutdown()

---

### SETUP SERIAL WebSocket SERVER to use RC Controller

Load **p5.serialcontrol** after connecting an Arduino device via USB

Select Port to Arduino and change const serialPort below accordingly

### Step 0.2 from README.md

---

Goto **Method 2** below to try the Virtual Joystick mouse control if you don't have a RC Controller

In [ ]:
# https://colab.research.google.com/drive/1TqF6RObllyslad3KrO4SiAuegRjuJDCF

from IPython.display import HTML, Audio
from google.colab import output

import cv2
import pygame
import time
import numpy as np
from google.colab.patches import cv2_imshow

steering = 0.0
throttle = 0.0
breaks = 0.0

pygame.init()
imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
newArray = np.copy(imageSeen)
print(newArray.shape)
newArray[:, :, [2, 0]] = newArray[:, :, [0, 2]]

C_html = """
<script src="https://cdn.jsdelivr.net/npm/p5@1.11.10/lib/p5.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/p5.serialserver@0.0.29/lib/p5.serialport.js"></script>
<script>

  // serial port name matching p5.serialcontrol selection
  const serialPort = 'COM3';
  // reference to the serial port
  let serial;
  let options = { baudRate: 115200 };
  let latestData = "waiting for data";

  alert('This is a JavaScript alert!');
  new p5();

  serial = new p5.SerialPort();

  serial.on('open', gotOpen);
  serial.on('data', gotData);
  serial.on('error', gotError);
  serial.on('close', gotClose);

  serial.open(serialPort, options);

  function gotOpen() {
    latestData = "Serial Port is Open";
    google.colab.kernel.invokeFunction('data_to_receive', [latestData], {});
  }

  function gotClose(){
    latestData = "Serial Port is Closed";
    google.colab.kernel.invokeFunction('data_to_receive', [latestData], {});
  }

  function gotError(theerror) {
    print(theerror);
  }

  function gotData() {
    let currentString = serial.readLine();
      trim(currentString);
    if (!currentString) return;
    latestData = currentString;
    google.colab.kernel.invokeFunction('line_to_receive', [latestData], {});
  }

</script>
"""

def handle_js_data(line):
  """
  This Python function will receive data from JavaScript.
  """
  #print(f"Received data from JavaScript: {line}")
  # Process the data as needed in your Python environment
  try:
      # Parse the comma-separated string
      var1_str, var2_str, var3_str = line.split(',')
      steering = float(var2_str) / 90
      throttle = int(var3_str) / 100

      # Print or process the variables
      #print(f"Read: var1={var1_str}, var2={steering}, var3={throttle}")
      imageSeen, x, y, z, speed, cte, lidar = gym.step(steering, throttle, breaks, False);
      newArray = np.copy(imageSeen)
      newArray[:, :, [2, 0]] = newArray[:, :, [0, 2]]

  except ValueError as e:
      print(f"Skipping malformed data: '{line}' Error: {e}")
  
  pygame.event.pump()
  #cv2_imshow(newArray)

display(HTML(C_html))
output.register_callback('line_to_receive', handle_js_data)


In [ ]:
print('speed: ', speed)
print('cte - Cross Track Error (how far from center line): ', cte)

#reset car if needed
gym.step(0.0, 0.0, 0.0, True)

#cv2.destroyWindow("image")

speed:  2.64904069900513
cte - Cross Track Error (how far from center line):  1.59317028522491
Resetting car...


---

### SETUP Web Serial to connect DUMBORC Transmitter via Arduino USB

---

There are 2 methods to control car and collect data.
===

- **Method 1** Keyboard control and post processing images.
- **Method 2** Click image to drive and collect data at the same time.

### Tip

- collect ~200 images minimum following on yellow lines.
- collect ~200 images minimum avoiding outer lines and cones.

Method 1 Keyboard control
===

### 2 windows will pop up:

- Live stream image from the 'virtual' camera.
- Pygame black square for keyboard inputs.

*Both windows are needed to control the car.*

### Controls
Click the **black square** to activate controls:

- Arrow keys: DRIVE
- C : CAMERA (take 3 pictures)
- X : BREAK
- Z : SHOW current car coordinates
- Red STOP button : 'kernel interrupt' to stop training loop.

### Images will save in *m1_images* folder.



In [ ]:
import cv2

def bgr8_to_jpeg(value, quality=100):
    return bytes(cv2.imencode('.jpg', value)[1])

import pygame, time
from pygame.locals import *
import numpy as np
import time
import os

counter = 1
steer = 0.0
throttle = 0.0
breaks = 0.0

pygame.init()
screen = pygame.display.set_mode((300, 200)) #black square
pygame.display.set_caption('Pygame Keyboard Test')
pygame.mouse.set_visible(0)

print ("\n\n************ CONTROLS ************\n\n   Arrow keys: DRIVE\n   Z : SHOW car coordinates\n   X : BREAK\n   C : CAMERA (take 5 pictures)\n   Red STOP button : 'kernel interrupt' to stop training loop\n\n***********************************\n\n")

dir = "m1_images"
try:
    os.makedirs(dir, exist_ok = False)
    print("Directory '%s' created successfully" % dir)
except OSError as error:
    print("Directory '%s' already exists" % dir)

imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
newArray = np.copy(imageSeen)
print(newArray.shape)
newArray[:, :, [2, 0]] = newArray[:, :, [0, 2]]

try:
    while True:
        keyState = pygame.key.get_pressed()
        if(len(keyState) != 0):
            if keyState[pygame.K_UP]:
                throttle = 0.5
            elif keyState[pygame.K_DOWN]:
                throttle = -1.0
            else:
                throttle = 0.0
            if keyState[pygame.K_RIGHT]:
                steer = 1.0
            elif keyState[pygame.K_LEFT]:
                steer = -1.0
            else:
                steer = 0.0

            if keyState[pygame.K_x]:
                breaks = 1000.0
            else:
                breaks = 0.0

            imageSeen, x, y, z, speed, cte, lidar = gym.step(steer, throttle, breaks, False);
            newArray = np.copy(imageSeen)
            newArray[:, :, [2, 0]] = newArray[:, :, [0, 2]]

            if keyState[pygame.K_c]:

                for i in range(3): #takes 3 pictures in one shot
                    cv2.imwrite('./m1_images/' + 'data_point_' + "{:03d}".format(counter) + ".jpg", (newArray))
                    counter = counter + 1
                print(str(counter-1) + " image(s) captured")
                time.sleep(.3)  #snap the photo and wait .3 seconds to allow finger to release button. While loop is too fast when you press B.
            if counter >= 999:
                print("TOO MANY PICTURES")
                gym.step(0.0, 0.0, 100.0, False)
                time.sleep(1)
                print('\nTRAINING STOPPED')
                gym.step(0.0, 0.0, 0.0, True);
                break

            if keyState[pygame.K_z]:
                print("x = "+str(x)+", z = "+str(z))
                time.sleep(.3)

            if keyState[pygame.K_v]:
                print(lidar)
                time.sleep(.3)

        else:
            throttle = 0.0
            steer = 0.0
            breaks = 0.0
            print('BRAKE')
            imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 100.0, False)

        pygame.event.pump()
        cv2.imshow("image", newArray)

except KeyboardInterrupt:

    #Resets steering and throttle to 0, applies breaks and resets for 1 second
    gym.step(0.0, 0.0, 100.0, False)
    time.sleep(1)
    print('\nTRAINING STOPPED')
    gym.step(0.0, 0.0, 0.0, True);



************ CONTROLS ************

   Arrow keys: DRIVE
   Z : SHOW car coordinates
   X : BREAK
   C : CAMERA (take 5 pictures)
   Red STOP button : 'kernel interrupt' to stop training loop

***********************************


Directory 'm1_images' already exists
(224, 224, 3)


---
### If finished - Click interrupt Jupyter kernel "STOP" button at top controls.
### Verify *m1_images* folder is populated with data. Exit this notebook.
### Proceed to *step 3.2* in README.md and run *postprocess_images.py* Good luck!

---

In [ ]:
gym.onShutdown()

Method 2 Image click and drive.
===

### 2 wisgets will pop up:

- Clickable 'virtual' camera that takes a picture and drives the car to that direction.
- Click and hold 'virtual' joystick to manually control the car.

*Both windows can control the car.*

### How to collect data
Click the **'virtual' camera image** in the direction you want the car to drive. Hold the click to pick up more speed. Click farther out to drive faster or closer to car to drive slower.
If you need to correct your position without taking a picture use the **'virtual' joystick** to manually move the car in position.

### Images will save in *m2_images* folder.


In [ ]:
try:
  import google.colab
  !pip install ipycanvas==0.11
  from google.colab import output
  output.enable_custom_widget_manager()
  IN_COLAB=True
except:
  IN_COLAB = False

In [ ]:
import ipywidgets
from ipycanvas import Canvas
from ipywidgets import Image
from ipywidgets import Layout
import cv2
import numpy as np
import os
from datetime import datetime


def handle_mouse_down(x, y):
    global newImage
    x = int(x)
    y = int(y)
    gym.step((2.0 * (x / 224.0 - 0.5)), (-2.0 * (y / 224.0 - 0.8)), 0.0, False)

    now = str(datetime.now())
    now = now.replace(" ", "")
    now = now.replace(".", "")
    now = now.replace(":", "-")
    newName = str(x) + "_" + str(y) + "_" + str(now) + ".jpg"
    cv2.imwrite("./m2_images/"+newName, newImage)

def handle_mouse_up(x, y):
    rewrite_canvas()

def rewrite_canvas():
    global newImage
    canvas.clear()
    imageSeen, cx, cy, cz, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
    newImage = np.copy(imageSeen)
    newImage[:, :, [2, 0]] = newImage[:, :, [0, 2]]
    cv2.imwrite("./m2_images/DELETE.jpg", newImage)
    current = Image.from_file('./m2_images/DELETE.jpg')
    canvas.draw_image(current, 0, 0)
    os.remove('./m2_images/DELETE.jpg')

def manual_mouse_down(x, y):
    joystick.on_mouse_move(manual_move, remove=False)

def manual_mouse_up(x, y):
    joystick.on_mouse_move(manual_move, remove=True)
    gym.step(0.0, 0.0, 1.0, False)

def manual_out(x, y):
    gym.step(0.0, 0.0, 1.0, False)

def manual_move(x, y):
    gym.step((2.0 * (x / 224.0 - 0.5)), (-2.0 * (y / 224.0 - 0.5)), 0.0, False)



global newImage
time.sleep(1)
gym.step(0.0, 0.0, 0.0, True);
time.sleep(1)

dir = "m2_images"
try:
    os.makedirs(dir, exist_ok = False)
    print("Directory '%s' created successfully" % dir)
except OSError as error:
    print("Directory '%s' already exists" % dir)

print ("\n\n************************ CONTROLS ************************\n\nRepeatedly click path on image to move car.\nClick farther out to drive faster or closer to car to drive slower.\nHINT: find the optimal driving path.\n\n***********************************************************\n\n")

canvas = Canvas(width=224, height=224)
rewrite_canvas()
canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_up(handle_mouse_up)

joystick = Canvas(width=224, height=224)
directions = Image.from_file('./jetsim-fork/jetsim_local/resources/joystick.png')
joystick.draw_image(directions, 0, 0)
joystick.on_mouse_up(manual_mouse_up)
joystick.on_mouse_down(manual_mouse_down)
joystick.on_mouse_move(manual_move, remove=True)
joystick.on_mouse_out(manual_out)
joystick.stroke_style = 'blue'
joystick.stroke_rect(2, 2, 222, 222)


all_controls = ipywidgets.HBox([canvas, joystick])
all_controls



In [ ]:
#reset car if needed
gym.step(0.0, 0.0, 0.0, True);

Resetting car...


---
### If finished - Verify *m2_images* folder is populated with data. Exit this notebook.
### Proceed to **step 4.1** in README.md and transfer folders to Google Drive. Good luck!

---

In [ ]:
gym.onShutdown()